In [4]:
import os
import pandas as pd
import mysql.connector

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Sh1v@ng1",
    database="ecommerce1"
)
cursor = conn.cursor()

# Helper function to map pandas dtypes to SQL
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return "INT"
    elif pd.api.types.is_float_dtype(dtype):
        return "FLOAT"
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return "DATETIME"
    else:
        return "TEXT"

# List of CSV files with table names
csv_files = [
    ("customers.csv", "customers"),
    ("geolocation.csv", "geolocation"),
    ("order_items.csv", "order_items"),
    ("orders.csv", "orders"),
    ("payments.csv", "payments"),
    ("products.csv", "products"),
    ("sellers.csv", "sellers"),
]

# Folder path
folder_path = "C:/Users/shiva/OneDrive/Desktop/ecommerce1"

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)

    # Load CSV
    df = pd.read_csv(file_path)

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Replace NaN with None for SQL NULL
    df = df.where(pd.notnull(df), None)

    # Drop and recreate table
    cursor.execute(f"DROP TABLE IF EXISTS `{table_name}`")
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert in batches
    batch_size = 1000
    cols = ', '.join([f'`{col}`' for col in df.columns])
    placeholders = ', '.join(['%s'] * len(df.columns))
    sql = f"INSERT INTO `{table_name}` ({cols}) VALUES ({placeholders})"

    data = [tuple(None if pd.isna(x) else x for x in row) for row in df.to_numpy()]

    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        cursor.executemany(sql, batch)
        conn.commit()

    print(f"✅ Finished importing {csv_file} into {table_name}")

# Close connection
cursor.close()
conn.close()


✅ Finished importing customers.csv into customers
✅ Finished importing geolocation.csv into geolocation
✅ Finished importing order_items.csv into order_items
✅ Finished importing orders.csv into orders
✅ Finished importing payments.csv into payments
✅ Finished importing products.csv into products
✅ Finished importing sellers.csv into sellers
